## Exercice
Faire un programme qui:

- Trouve les 10 villes de France les + peuplées (e.g https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées)
- Pour chaque ville, trouve sa distance avec les autres.
 - par exemple, avec google maps api (nécessite de s'authentifier / créer un token) avec https://github.com/googlemaps/google-maps-services-python (cf. doc token dans le readme)
 - ou avec https://fr.distance24.org/ (pas besoin de s'authentifier)
- Trouve les villes les plus proches


In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [39]:
def get_top_cities(n):
    page_wiki = requests.get('https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées')
    soup = BeautifulSoup(page_wiki.text)

    cities = []

    for line in soup.find('table').find('tbody').findAll('td',{'style':'text-align:left'}) :
        city = line.findAll('b')
        for el in city:
            cities.append(el.find('a').text)
    
    return cities[:n]

In [41]:
get_top_cities(10)

['Paris',
 'Marseille',
 'Lyon',
 'Toulouse',
 'Nice',
 'Nantes',
 'Montpellier',
 'Strasbourg',
 'Bordeaux',
 'Lille']

In [129]:
requests.get('https://fr.distance24.org/route.json?stops=Lille|Paris').json()

{'distances': [204],
 'distance': 204,
 'stops': [{'travelGuides_timestamp': '2013-02-09T11:01:35+00:00',
   'locatedBy_version': 7,
   'locatedBy_timestamp': '2012-10-22T13:01:20+00:00',
   'city': 'Lille',
   'travelGuides': [{'thumbnail': 'http://ecx.images-amazon.com/images/I/51f6PW84BsL._SL160_.jpg',
     'title': 'Guide historique et architectural de Lille',
     'url': 'http://www.amazon.fr/Guide-historique-architectural-Lille-darchitecture/dp/2847420398%3FSubscriptionId%3D1ERC8D3Q83S91BW8KGR2%26tag%3Dreezom-21%26linkCode%3Dxm2%26camp%3D2025%26creative%3D165953%26creativeASIN%3D2847420398'},
    {'thumbnail': 'http://ecx.images-amazon.com/images/I/517o%2BaU446L._SL160_.jpg',
     'title': 'Lille',
     'url': 'http://www.amazon.fr/Lille-DUVIVIER-Isabelle/dp/2755800615%3FSubscriptionId%3D1ERC8D3Q83S91BW8KGR2%26tag%3Dreezom-21%26linkCode%3Dxm2%26camp%3D2025%26creative%3D165953%26creativeASIN%3D2755800615'},
    {'thumbnail': 'http://ecx.images-amazon.com/images/I/61OpIdOWdOL._SL16

In [66]:
def get_distance (city1, city2):
    dist = requests.get('https://fr.distance24.org/route.json?stops={city1}|{city2}'.format(city1=city1,city2=city2)).json()
    return dist['distance']

In [90]:
distances = {}
top_10 = get_top_cities(10)
for city1 in top_10:
    for city2 in top_10:
        if city1 != city2:
            distances[(city1,city2)] = get_distance(city1,city2)

In [120]:
df = pd.DataFrame(distances.keys(),columns=['origin','destination'])
df['distances']=distances.values()

In [121]:
df.sort_values('distances')

,origin,destination,distances
55,Montpellier,Marseille,126
14,Marseille,Montpellier,126
12,Marseille,Nice,159
37,Nice,Marseille,159
32,Toulouse,Montpellier,196
57,Montpellier,Toulouse,196
81,Lille,Paris,204
8,Paris,Lille,204
34,Toulouse,Bordeaux,212
75,Bordeaux,Toulouse,212
